In [1]:
import sys

sys.path.append("../")
from evaluation.evaluate import (
    get_froc_vals,
    convert_mm_to_pixel,
    get_F1_scores,
)
from monkey.data.data_utils import open_json_file, extract_id
import os
from pprint import pprint

/home/u1910100/miniconda3/envs/tiatoolbox/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
GROUND_TRUTH_DIRECTORY = (
    "/home/u1910100/Downloads/Monkey/annotations/json"
)
FOLD = 1
model_name = "convnext_tiny_pannuke_256"
PREDICT_DIR = f"/home/u1910100/Documents/Monkey/local_output/{model_name}/Fold_{FOLD}"

SPACING_LEVEL0 = 0.24199951445730394

split_info = open_json_file(
    "/home/u1910100/Documents/Monkey/patches_256/wsi_level_split.json"
)

val_wsi_files = split_info[f"Fold_{FOLD}"]["test_files"]

inflamm_sum_score = 0.0
inflamm_sum_f1 = 0.0
lymph_sum_score = 0.0
lymph_sum_f1 = 0.0
mono_sum_score = 0.0
mono_sum_f1 = 0.0

for wsi_name in val_wsi_files:
    wsi_name_without_ext = os.path.splitext(wsi_name)[0]
    wsi_id = extract_id(wsi_name)

    gt_inf_cells = open_json_file(
        json_path=os.path.join(
            GROUND_TRUTH_DIRECTORY,
            f"{wsi_id}_inflammatory-cells.json",
        )
    )
    gt_lymphocytes = open_json_file(
        json_path=os.path.join(
            GROUND_TRUTH_DIRECTORY, f"{wsi_id}_lymphocytes.json"
        )
    )
    gt_monocytes = open_json_file(
        json_path=os.path.join(
            GROUND_TRUTH_DIRECTORY, f"{wsi_id}_monocytes.json"
        )
    )

    location_detected_inflamm = os.path.join(
        PREDICT_DIR, f"{wsi_id}_detected-inflammatory-cells.json"
    )
    result_detected_inflamm = open_json_file(
        json_path=location_detected_inflamm,
    )
    result_detected_inflamm = convert_mm_to_pixel(
        result_detected_inflamm
    )

    location_detected_lymphocytes = os.path.join(
        PREDICT_DIR, f"{wsi_id}_detected-lymphocytes.json"
    )
    result_detected_lymphocytes = open_json_file(
        json_path=location_detected_lymphocytes,
    )
    result_detected_lymphocytes = convert_mm_to_pixel(
        result_detected_lymphocytes
    )

    location_detected_monocytes = os.path.join(
        PREDICT_DIR, f"{wsi_id}_detected-monocytes.json"
    )
    result_detected_monocytes = open_json_file(
        json_path=location_detected_monocytes,
    )
    result_detected_monocytes = convert_mm_to_pixel(
        result_detected_monocytes
    )

    inflamm_froc = get_froc_vals(
        gt_inf_cells,
        result_detected_inflamm,
        radius=int(5 / SPACING_LEVEL0),
    )

    inflamm_f1 = get_F1_scores(
        gt_inf_cells,
        result_detected_inflamm,
        radius=int(5 / SPACING_LEVEL0),
    )

    lymph_froc = get_froc_vals(
        gt_lymphocytes,
        result_detected_lymphocytes,
        radius=int(4 / SPACING_LEVEL0),
    )

    lymph_f1 = get_F1_scores(
        gt_lymphocytes,
        result_detected_lymphocytes,
        radius=int(4 / SPACING_LEVEL0),
    )

    mono_froc = get_froc_vals(
        gt_monocytes,
        result_detected_monocytes,
        radius=int(5 / SPACING_LEVEL0),
    )

    mono_f1 = get_F1_scores(
        gt_monocytes,
        result_detected_monocytes,
        radius=int(5 / SPACING_LEVEL0),
    )

    # pprint(wsi_id)
    # pprint(f"FROC score: {inflamm_froc['froc_score_slide']}")
    inflamm_sum_score += inflamm_froc["froc_score_slide"]
    inflamm_sum_f1 += inflamm_f1["F1"]

    lymph_sum_score += lymph_froc["froc_score_slide"]
    lymph_sum_f1 += lymph_f1["F1"]

    mono_sum_score += mono_froc["froc_score_slide"]
    mono_sum_f1 += mono_f1["F1"]

pprint(
    f"Average Inflamm FROC = {inflamm_sum_score / len(val_wsi_files)}"
)
pprint(f"Average Inflamm F1 = {inflamm_sum_f1 / len(val_wsi_files)}")
pprint(
    f"Average Lymphocytes FROC = {lymph_sum_score / len(val_wsi_files)}"
)
pprint(
    f"Average Lymphocytes F1 = {lymph_sum_f1 / len(val_wsi_files)}"
)
pprint(
    f"Average Monocytes FROC = {mono_sum_score / len(val_wsi_files)}"
)
pprint(f"Average Monocytes F1 = {mono_sum_f1 / len(val_wsi_files)}")

'Average Inflamm FROC = 0.32435014352875235'
'Average Inflamm F1 = 0.7523950339604142'
'Average Lymphocytes FROC = 0.3630850329248785'
'Average Lymphocytes F1 = 0.6930402723310959'
'Average Monocytes FROC = 0.21923917414408006'
'Average Monocytes F1 = 0.4877555846399448'


In [17]:
SPACING_LEVEL0 = 0.24199951445730394

GT_MM = True
if GT_MM:
    GROUND_TRUTH_DIRECTORY = (
        "/home/u1910100/Downloads/Monkey/annotations/json_mm"
    )
else:
    GROUND_TRUTH_DIRECTORY = (
        "/home/u1910100/Downloads/Monkey/annotations/json_pixel"
    )
radius_lymph = 0.004 if GT_MM else int(4 / SPACING_LEVEL0)
radius_mono = 0.005 if GT_MM else int(5 / SPACING_LEVEL0)
radius_infl = 0.005 if GT_MM else int(5 / SPACING_LEVEL0)

wsi_id = "A_P000002"
PREDICT_DIR = f"/home/u1910100/Documents/Monkey/test/output"

SPACING_LEVEL0 = 0.24199951445730394


inflamm_sum_score = 0.0
inflamm_sum_f1 = 0.0
lymph_sum_score = 0.0
lymph_sum_f1 = 0.0
mono_sum_score = 0.0
mono_sum_f1 = 0.0


gt_inf_cells = open_json_file(
    json_path=os.path.join(
        GROUND_TRUTH_DIRECTORY,
        f"{wsi_id}_inflammatory-cells.json",
    )
)
gt_lymphocytes = open_json_file(
    json_path=os.path.join(
        GROUND_TRUTH_DIRECTORY, f"{wsi_id}_lymphocytes.json"
    )
)
gt_monocytes = open_json_file(
    json_path=os.path.join(
        GROUND_TRUTH_DIRECTORY, f"{wsi_id}_monocytes.json"
    )
)

location_detected_inflamm = os.path.join(
    PREDICT_DIR, f"detected-inflammatory-cells.json"
)
result_detected_inflamm = open_json_file(
    json_path=location_detected_inflamm,
)
if not GT_MM:
    result_detected_inflamm = convert_mm_to_pixel(
        result_detected_inflamm
    )

location_detected_lymphocytes = os.path.join(
    PREDICT_DIR, f"detected-lymphocytes.json"
)
result_detected_lymphocytes = open_json_file(
    json_path=location_detected_lymphocytes,
)
if not GT_MM:
    result_detected_lymphocytes = convert_mm_to_pixel(
        result_detected_lymphocytes
    )

location_detected_monocytes = os.path.join(
    PREDICT_DIR, f"detected-monocytes.json"
)
result_detected_monocytes = open_json_file(
    json_path=location_detected_monocytes,
)
if not GT_MM:
    result_detected_monocytes = convert_mm_to_pixel(
        result_detected_monocytes
    )

inflamm_froc = get_froc_vals(
    gt_inf_cells,
    result_detected_inflamm,
    radius=radius_infl,
)

inflamm_f1 = get_F1_scores(
    gt_inf_cells,
    result_detected_inflamm,
    radius=radius_infl,
)

lymph_froc = get_froc_vals(
    gt_lymphocytes,
    result_detected_lymphocytes,
    radius=radius_lymph,
)

lymph_f1 = get_F1_scores(
    gt_lymphocytes,
    result_detected_lymphocytes,
    radius=radius_lymph,
)

mono_froc = get_froc_vals(
    gt_monocytes,
    result_detected_monocytes,
    radius=radius_mono,
)

mono_f1 = get_F1_scores(
    gt_monocytes,
    result_detected_monocytes,
    radius=radius_mono,
)

# pprint(wsi_id)
# pprint(f"FROC score: {inflamm_froc['froc_score_slide']}")
inflamm_sum_score += inflamm_froc["froc_score_slide"]
inflamm_sum_f1 += inflamm_f1["F1"]

lymph_sum_score += lymph_froc["froc_score_slide"]
lymph_sum_f1 += lymph_f1["F1"]

mono_sum_score += mono_froc["froc_score_slide"]
mono_sum_f1 += mono_f1["F1"]

pprint(f"Average Inflamm FROC = {inflamm_sum_score} ")
pprint(f"Average Lymphocytes FROC = {lymph_sum_score}")
pprint(f"Average Monocytes FROC = {mono_sum_score}")

'Average Inflamm FROC = 0.31237995460101275 '
'Average Lymphocytes FROC = 0.372236727589208'
'Average Monocytes FROC = 0.3110877561744613'


In [4]:
print(inflamm_froc["fp_probs_slide"])
print(inflamm_froc["tp_probs_slide"])
print(inflamm_froc["total_pos_slide"])

[0.9473825693130493, 0.9407113790512085, 0.8476263880729675, 0.8388563394546509, 0.7760008573532104, 0.5724475979804993, 0.526828408241272, 0.9908714294433594, 0.9907762408256531, 0.9892728924751282, 0.9891304969787598, 0.9888019561767578, 0.9859879612922668, 0.98491370677948, 0.9848983883857727, 0.9817803502082825, 0.9815008044242859, 0.9796956181526184, 0.9788524508476257, 0.9681147336959839, 0.9664894342422485, 0.9646714329719543, 0.9640976786613464, 0.9597177505493164, 0.957474410533905, 0.9476453065872192, 0.9463480710983276, 0.9279845952987671, 0.9114087224006653, 0.9108670353889465, 0.9091697931289673, 0.9086969494819641, 0.90056973695755, 0.8934836387634277, 0.8920847773551941, 0.8845203518867493, 0.8843874931335449, 0.8834713101387024, 0.8834012746810913, 0.8803099393844604, 0.8680601119995117, 0.8657827377319336, 0.8502537608146667, 0.8484203219413757, 0.8246135711669922, 0.8227517008781433, 0.8187078237533569, 0.8106076121330261, 0.7597745060920715, 0.7537431716918945, 0.731